# Hausaufgabe
translate(Hausaufgabe) -> **Homework**

Halo!
На семинаре мы создали простую seq2seq модель на основе rnn для перевода, а сейчас постараемся засунуть туда attention. Работать будем с тем же датасетом DE->EN (датасеты получше просто не влезают в память колаба, но если у вас есть CPU+тонна времени или GPU побольше, то можно попробовать построить перевод на WMT14 или IWSLT )

В конце домашней работы предполагается написание отчета о проделанной работе.

In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import spacy

import random
import math
import time

from torchtext.datasets import TranslationDataset, Multi30k #WMT14, IWSLT
from torchtext.data import Field, BucketIterator

import torch.nn.functional as F

In [2]:
seed = 43

random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# ! python -m spacy download en
# ! python -m spacy download de


spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [4]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

# немецкий язык является полем SRC, а английский в поле TRG
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [5]:
# В датасете содержится ~ 30к предложений средняя длина которых 11
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'),  fields = (SRC, TRG))

Давайте посмотрим что у нас с датасетом и сделаем словари для SRC и TGT

In [6]:
labels = ['train', 'validation', 'test']
dataloaders = [train_data, valid_data, test_data]
for d, l in zip(dataloaders, labels):
    print("Number of sentences in {} : {}".format(l, len(d.examples)))


Number of sentences in train : 29000
Number of sentences in validation : 1014
Number of sentences in test : 1000


In [7]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)
print("Number of words in source vocabulary", len(SRC.vocab))
print("Number of words in source vocabulary", len(TRG.vocab))

Number of words in source vocabulary 7855
Number of words in source vocabulary 5893


## Encoder

Энкодер будет ровно как в семинаре, с кдинственным изменением -- forward будет возвращать не только hidden, cell, но еще и outputs. Это нужно (надеюсь, вы уже поняли) для использования attention в декодере

In [8]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        """
        :param: input_dim is the size/dimensionality of the one-hot vectors that will be input to the encoder. This is equal to the input (source) vocabulary size.
        :param: emb_dim is the dimensionality of the embedding layer. This layer converts the one-hot vectors into dense vectors with emb_dim dimensions.
        :param: hid_dim is the dimensionality of the hidden and cell states.
        :param: n_layers is the number of layers in the RNN.
        :param: percentage of the dropout to use
        
        """
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(self.emb_dim, self.hid_dim, self.n_layers)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        """
        :param: src sentences (src_len x batch_size)
        """
        embedded = self.embedding(src)
        embedded = self.dropout(embedded)
        outputs, (hidden, cell) = self.rnn(embedded)
        return outputs, hidden, cell

## Decoder
Оп ля, а тут уже что-то новенькое

Мы будем реализовывать attention, который будет смотреть из tgt в src (НЕ self-attention). 

Определим два класса -- Attention и DecoderAttn. Мы разбили их на два класса, чтобы можно было играться с типом внимания, не меняя код DecoderAttn. Как вы помните с лекции, в качестве аттеншена можно брать любую странную функцию (конкатенация, маленькая сеточка, ...), и все будет работать! Поэтому вам предлагается попробовать несколько разных.


---------------------
Есть два подхода к реализации аттеншена:

Подход #1:

1. Вычисляется embed
2. На основе hidden c прошлого шага, embedded и (возможно) enc_out вычисляется attention, а точнее, веса attention (поэтому не забудьте softmax!!). Размерность batch_size * max_len, max_len -- максимальная длина предложения в батче, т.е. shape[0] от выхода энкодера.
3. К enc_out применяется attention: чаще всего dot product от enc_out и attention_weights (не забудьте про измерение батч. Чтобы нормально вычислить dot_product по батчу, вам поможет torch.bmm)
4. Берутся attention и embedded и сворачиваются в один вектор размерности такой, чтобы кормить его self.lstm. Например, это можно сделать с помощью обычного линейного слоя
5. Вычисляется новое скрытое состояние new_hidden. Это наша self.lstm, примененная к выходу пункта 4.
6. Вычисляется prediction, как в семинаре

Грубо говоря, вся разница с семинаром в том, что мы вместо того, чтобы embedded пихать в self.lstm, миксуем аттэншен на основе всего, что имеем (enc_out, hidden, embedded) и запихиваем в self.lstm микс аттэншена и embedded.

![alt text](https://i.imgur.com/cmkRY0r.png)


Подход #2:

1. Вычисляется embed
2. Вычисляется output, new_hidden (строчка output, (hidden, cell) = self.rnn(embedded, (hidden, cell)))
3. На основе output и enc_out вычисляется attention, а точнее, веса attention (поэтому не забудьте softmax!!)
3. К enc_out применяется attention: чаще всего dot product от enc_out и attention_weights (не забудьте про измерение батч. Чтобы нормально вычислить dot_product по батчу, вам поможет torch.bmm)
4. Вычисляется prediction на основе attention и output. Можно, например, взять nn.Linear() от конкатенации attention и output.

Разница с первым подходом в том, что мы сначала вычисляем выход rnn слоя, а потом смотрим вниманием на src и на основе выхода rnn и attn считаем выход (prediction). 

![alt text](https://i.imgur.com/5aWjQWv.png)


Вам предлагается реализовать хотя бы 1 из вариантов и хотя бы 2 варианта функции attention (в классе Attention)


In [16]:
class Attention(nn.Module):
    def __init__(self, batch_size, hidden_dim, method="one-layer-net"):
        super(Attention, self).__init__()
        self.method = method
        self.hidden_dim = hidden_dim
        
        if method == "one-layer-net":
            self.fc = nn.Linear(hidden_dim, hidden_dim, bias=False)
        elif method == "concat":
            self.fc = nn.Linear(hidden_dim, hidden_dim, bias=False)
            self.weight = nn.Parameter(torch.FloatTensor(hidden_dim,1))
        
    def forward(self, last_hidden, encoder_outputs, seq_len=None):
        if self.method == "one-layer-net":
            out = self.fc(last_hidden)
            return torch.bmm(encoder_outputs.permute(1, 0, 2), out.permute(1, 2, 0)).squeeze(-1).permute(1,0)
        elif self.method == "concat":
            out = torch.tanh(self.fc(last_hidden+encoder_outputs))
            out = out.permute(1, 0, 2)
            return out.matmul(self.weight).squeeze(-1).permute(1,0)


class DecoderAttn(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, attention, dropout=0.1):
        super(DecoderAttn, self).__init__()
        
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        
        self.attn = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(self.emb_dim, self.hid_dim, self.n_layers) #(lstm embd, hid, layers, dropout)
        self.out = nn.Linear(self.hid_dim*2, self.output_dim) # Projection :hid_dim x output_dim
        self.dropout = nn.Dropout(dropout)

        
    def forward(self, input, hidden, cell, encoder_output):
        
        input = input.unsqueeze(0)
        
        embedded = self.embedding(input)
        embedded = self.dropout(embedded)
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        alignment_scores = self.attn(output, encoder_output)
        attn_weights = F.softmax(alignment_scores, dim = 0)
        context_vector = torch.bmm(attn_weights.unsqueeze(0).permute(2,0,1), encoder_output.permute(1,0,2)).permute(1,0,2)
        output = torch.cat((output, context_vector),-1)
        prediction = self.out(output[0])
        
        return prediction, hidden, cell

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Seq2Seq module

Здесь опять ничего не поменяется кроме того, что энкодер теперь возвращает свой output, а декодер его принимает

In [17]:
BOS_IDX = SRC.vocab.stoi['<sos>']

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self._init_weights() 
        self.max_len=30
    
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        """
        :param: src (src_len x batch_size)
        :param: tgt
        :param: teacher_forcing_ration : if 0.5 then every second token is the ground truth input
        """
        
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        enc_out, hidden, cell = self.encoder(src)
        
        input = trg[0,:]
        
        for t in range(1, max_len):
            
            output, hidden, cell = self.decoder(input, hidden, cell, enc_out)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            input = (trg[t] if teacher_force else top1)
        
        return outputs
    
    def translate(self, src):
        trg_vocab_size = self.decoder.output_dim
        
        outputs = []
        src = torch.tensor(src).to(self.device)
        sent_vec = SRC.process([src]).to(device)
        sent_vec = src.permute(1,0)
        enc_out, hidden, cell = self.encoder(sent_vec)
        
        input = torch.tensor([BOS_IDX]).to(self.device)

        for t in range(1, self.max_len):
            
            output, hidden, cell = self.decoder(input, hidden, cell, enc_out)
            top1 = output.max(1)[1]
            outputs.append(top1)
            input = (top1)
        return outputs
    
    def _init_weights(self):
        p = 0.08
        for name, param in self.named_parameters():
            nn.init.uniform_(param.data, -p, p)
        


In [18]:
input_dim = len(SRC.vocab)
output_dim = len(TRG.vocab)
src_embd_dim =  tgt_embd_dim = 256
hidden_dim = 512
num_layers =  2
dropout_prob = 0.2

batch_size = 64
PAD_IDX = TRG.vocab.stoi['<pad>']

iterators = BucketIterator.splits((train_data, valid_data, test_data),
                                  batch_size = batch_size, device = device)
train_iterator, valid_iterator, test_iterator = iterators

atn = Attention(batch_size, hidden_dim, method="concat")
enc = Encoder(input_dim, src_embd_dim, hidden_dim, num_layers, dropout_prob)
dec = DecoderAttn(output_dim, tgt_embd_dim, hidden_dim, num_layers, atn, dropout_prob)
model = Seq2Seq(enc, dec, device).to(device)


In [19]:
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): DecoderAttn(
    (attn): Attention(
      (fc): Linear(in_features=512, out_features=512, bias=False)
    )
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2)
    (out): Linear(in_features=1024, out_features=5893, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
)

In [20]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [21]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing !!
            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)


            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [22]:
max_epochs = 10
CLIP = 1

# TODO
optimizer = optim.AdamW(model.parameters(), lr = 1e-3)
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

best_valid_loss = float('inf')

for epoch in range(max_epochs):
    
    
    train_loss = round(train(model, train_iterator, optimizer, criterion, CLIP), 5)
    valid_loss = round(evaluate(model, valid_iterator, criterion),5)
    
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print('Epoch: {} \n Train Loss {}  Val loss {}:'.format(epoch, train_loss, valid_loss))
    print('Train Perplexity {}  Val Perplexity {}:'.format(np.exp(train_loss), np.exp(valid_loss)))


Epoch: 0 
 Train Loss 4.5697  Val loss 4.28349:
Train Perplexity 96.51515088396333  Val Perplexity 72.49299960309547:
Epoch: 1 
 Train Loss 3.35817  Val loss 3.64849:
Train Perplexity 28.736554836602078  Val Perplexity 38.41661314427171:
Epoch: 2 
 Train Loss 2.73932  Val loss 3.39174:
Train Perplexity 15.476457526253919  Val Perplexity 29.717615961695998:
Epoch: 3 
 Train Loss 2.34848  Val loss 3.31621:
Train Perplexity 10.469643765541248  Val Perplexity 27.555716237187013:
Epoch: 4 
 Train Loss 2.06601  Val loss 3.3209:
Train Perplexity 7.893266071782104  Val Perplexity 27.68525607982288:
Epoch: 5 
 Train Loss 1.84076  Val loss 3.38189:
Train Perplexity 6.301325449006046  Val Perplexity 29.426334361423248:
Epoch: 6 
 Train Loss 1.63626  Val loss 3.28277:
Train Perplexity 5.135925187577853  Val Perplexity 26.649489641080965:
Epoch: 7 
 Train Loss 1.46783  Val loss 3.39328:
Train Perplexity 4.3398075334603545  Val Perplexity 29.763416347522412:
Epoch: 8 
 Train Loss 1.34174  Val loss 3

In [31]:
input_dim = len(SRC.vocab)
output_dim = len(TRG.vocab)
src_embd_dim =  tgt_embd_dim = 256
hidden_dim = 512
num_layers =  2
dropout_prob = 0.2

batch_size = 64
PAD_IDX = TRG.vocab.stoi['<pad>']

iterators = BucketIterator.splits((train_data, valid_data, test_data),
                                  batch_size = batch_size, device = device)
train_iterator, valid_iterator, test_iterator = iterators

atn = Attention(batch_size, hidden_dim, method="one-layer-net")
enc = Encoder(input_dim, src_embd_dim, hidden_dim, num_layers, dropout_prob)
dec = DecoderAttn(output_dim, tgt_embd_dim, hidden_dim, num_layers, atn, dropout_prob)
model = Seq2Seq(enc, dec, device).to(device)


In [263]:
max_epochs = 10
CLIP = 1

# TODO
optimizer = optim.AdamW(model.parameters(), lr = 5e-4)
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

best_valid_loss = float('inf')

for epoch in range(max_epochs):
    
    
    train_loss = round(train(model, train_iterator, optimizer, criterion, CLIP), 5)
    valid_loss = round(evaluate(model, valid_iterator, criterion),5)
    
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print('Epoch: {} \n Train Loss {}  Val loss {}:'.format(epoch, train_loss, valid_loss))
    print('Train Perplexity {}  Val Perplexity {}:'.format(np.exp(train_loss), np.exp(valid_loss)))


Epoch: 0 
 Train Loss 4.84992  Val loss 4.64837:
Train Perplexity 127.73017102359952  Val Perplexity 104.41465091108981:
Epoch: 1 
 Train Loss 3.81275  Val loss 4.1137:
Train Perplexity 45.27477345490371  Val Perplexity 61.17263811991019:
Epoch: 2 
 Train Loss 3.27865  Val loss 3.86091:
Train Perplexity 26.539919613007946  Val Perplexity 47.5085644975907:
Epoch: 3 
 Train Loss 2.96432  Val loss 3.7336:
Train Perplexity 19.38151932630632  Val Perplexity 41.829423358429345:
Epoch: 4 
 Train Loss 2.72551  Val loss 3.66943:
Train Perplexity 15.264196682395824  Val Perplexity 39.229538649224885:


In [23]:
test_loss = evaluate(model, test_iterator, criterion)

print('| Test Loss: {} Test PPL:{}|'.format(test_loss, np.exp(test_loss)))

| Test Loss: 3.490470603108406 Test PPL:32.80138050621148|


In [24]:
EOS_IDX = SRC.vocab.stoi['<eos>']

def translate(sentence):
    """
    function that uses .translate() method of the model to translate german sentence into english
    params:
        sentence: tokenized gernam sentence
    """
    out = []
#     sentence = sentence.lower()
    sent_vec = [BOS_IDX] + [SRC.vocab.stoi[token] for token in sentence] + [EOS_IDX]
    sent_vec += [PAD_IDX]*(30-len(sent_vec))
    translation_idx = model.translate(torch.tensor([sent_vec]))
    for t in translation_idx:
        if t[0] != EOS_IDX:
#             print(TRG.vocab.itos[t[0]], end=' ')
            out.append(TRG.vocab.itos[t[0]])
        else:
            break
    return out

In [25]:
translate("ein klein apfel".split())

/home/care1e55/progz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


['an', 'adult', 'of', 'young', 'children', '.']

In [26]:
translate("stiefeln schwarzen und kleid rot-weißen einem in frau einer mit straße der auf fotoshooting ein".split())

/home/care1e55/progz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


['a',
 'photo',
 'shoot',
 'with',
 'a',
 'woman',
 'in',
 'a',
 'red',
 'and',
 'red',
 'dress',
 'and',
 'black',
 'boots',
 'on',
 'the',
 'street',
 '.']

ИИИИ давайте также научимся считать самую популярную метрику для перевода -- BLEU (https://en.wikipedia.org/wiki/BLEU)

В общем-то, вам повезло -- ее писать руками скучно, да и nltk ее написало за вас:

In [27]:
from nltk.translate.bleu_score import corpus_bleu
def compute_bleu(inp_lines, out_lines):
    """ Estimates corpora-level BLEU score of model's translations given inp and reference out """
    translations = [translate(line) for line in inp_lines]
    return corpus_bleu([[ref] for ref in out_lines], translations) * 100

In [28]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'),  fields = (SRC, TRG))

In [ ]:
compute_bleu([i.src for i in train_data], [j.trg for j in train_data]) * 100

In [29]:
compute_bleu([i.src for i in valid_data], [j.trg for j in valid_data])

/home/care1e55/progz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


30.96331779698981

In [30]:
compute_bleu([i.src for i in test_data], [j.trg for j in test_data])

/home/care1e55/progz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


32.067326562529466

Если вы реализовали несколько методов аттеншена, опишите каждый из них и полученную метрику на нем в отчете.

В реализации использовались материалы семинара и статья https://blog.floydhub.com/attention-mechanism/#luong-att-step2
Реализован способ 2 и 2 вида подсчета score attention - concat и one-layer (general)
В one-layer hidden декодера передается в линейный слой - формируется вектор весов размерности hidden который будут весам для encoder output когда мы их меремножим. Самом перемножение делается по всему батчу для чего приходилось сделать permute чтобы размерности батчей были по нулевой оси в соответвии со спецификацией метода bmm. Берем от этого softmax и получили attention по всему батчу. Далее это также тензорно перемножается (опять по батчу с permute) с выходом энкодера получая новый взвешенный вектор контекста. Output с контекстом конкатенируются и через линейный слой размерности словаря применяется softmax для предсказания наиболее верятного слова.
В concat вектора decoder_hidden и encoder_output складываются и применяется линейный слой размерности hidden и с активацией tanh - выход матрично (побатчево) перемножается с весами. Остальное аналогично.
Переводы слов из тренировочной выборки переводятся примерно похоже, в отличии от слов которых модель могла не видеть. В методе транслате в качестве input передается символ начала предложения как стартовый и далее слова предсказываются по контектсу с attention.
В ходе обучения для каждого из методов получились примерно одинаковые PPL ~30, loss ~3.5 и BLEU ~ 30